In [1]:
import pandas as pd
from assign_unique_ids_functions import *
import numpy as np

In [2]:
df = pd.read_csv('../input/awards.csv.gz')

/Users/invinst/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,PPS.Award.Detail.ID,Requester.Full.Name,Current.Star,Incident.Start.Date,Resignation.Date,Incident.End.Date,Award.Type,Birth.Year,Race,Gender,...,Tracking.NO,Last.Promotion.Date,Award.Request.Date,Appointed.Date,Rank,First.Name,Last.Name,Middle.Initial,Middle.Initial2,Suffix.Name
0,8038267,"OCHOA, PETER",1424.0,2008-07-25,NaN,2008-07-25,HONORABLE MENTION,1971.0,WHITE,MALE,...,NaN,NaN,2008-07-27,2005-09-26,9171-SERGEANT OF POLICE,JEFFERY,AARON,M,NaN,NaN
1,8046162,"MAZUR, SANDY",1424.0,2005-09-26,NaN,2008-09-26,ATTENDANCE RECOGNITION AWARD,1971.0,WHITE,MALE,...,NaN,NaN,2008-12-20,2005-09-26,9171-SERGEANT OF POLICE,JEFFERY,AARON,M,NaN,NaN
2,8046771,"NIEVES, ROBERTO",1424.0,2009-01-03,NaN,2009-01-03,HONORABLE MENTION,1971.0,WHITE,MALE,...,NaN,NaN,2009-01-04,2005-09-26,9171-SERGEANT OF POLICE,JEFFERY,AARON,M,NaN,NaN
3,8067659,"TUNSTALL, TRACY",1424.0,2010-09-21,NaN,2010-09-21,HONORABLE MENTION,1971.0,WHITE,MALE,...,NaN,NaN,2010-09-27,2005-09-26,9171-SERGEANT OF POLICE,JEFFERY,AARON,M,NaN,NaN
4,8074425,"DAVIES, TRACY",1424.0,2011-06-03,NaN,2011-06-03,HONORABLE MENTION,1971.0,WHITE,MALE,...,NaN,NaN,2011-06-10,2005-09-26,9171-SERGEANT OF POLICE,JEFFERY,AARON,M,NaN,NaN


In [4]:
df.columns

Index(['PPS.Award.Detail.ID', 'Requester.Full.Name', 'Current.Star',
       'Incident.Start.Date', 'Resignation.Date', 'Incident.End.Date',
       'Award.Type', 'Birth.Year', 'Race', 'Gender', 'Ceremony.Date',
       'Current.Award.Status', 'Tracking.NO', 'Last.Promotion.Date',
       'Award.Request.Date', 'Appointed.Date', 'Rank', 'First.Name',
       'Last.Name', 'Middle.Initial', 'Middle.Initial2', 'Suffix.Name'],
      dtype='object')

In [8]:
id_cols = ["First.Name", "Last.Name", "Appointed.Date", "Gender"]
compare_cols = ["Middle.Initial", "Middle.Initial2", "Resignation.Date", "Current.Star", "Birth.Year"]
uid = 'awards_ID'

In [9]:
dfu = df[id_cols + compare_cols].drop_duplicates()
print(dfu.shape)
dfu.head()

(32979, 9)


,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Middle.Initial2,Resignation.Date,Current.Star,Birth.Year
0,JEFFERY,AARON,2005-09-26,MALE,M,NaN,NaN,1424.0,1971.0
6,KARINA,AARON,2005-09-26,FEMALE,NaN,NaN,NaN,20373.0,1980.0
12,SUSAN,AARUP,2009-01-01,FEMALE,J,NaN,2009-02-27,NaN,1969.0
21,ANTHONY,ABBATE,1994-12-05,MALE,G,NaN,2009-12-22,NaN,1968.0
22,CARMEN,ABBATE,1954-10-16,MALE,S,NaN,1993-06-04,NaN,1930.0


In [12]:
dfu[dfu['Middle.Initial2'].notnull()]

,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Middle.Initial2,Resignation.Date,Current.Star,Birth.Year
11998,TONI,BRENNAN,1982-05-03,FEMALE,A,J,1985-11-25,NaN,1960.0
37799,MAGNUM,FLORES,1995-12-04,MALE,Y,M,2017-01-15,NaN,1957.0
45610,RICHARD,GRAND,1986-06-16,MALE,A,J,2011-11-15,NaN,1956.0
85687,PAUL,MOSELEY,1978-04-03,FEMALE,A,M,1982-06-05,NaN,1943.0
95980,BRIAN,PENNY,1985-11-18,MALE,A,P,2010-11-29,NaN,1954.0
115896,GEORGE,SMITH,1968-03-04,MALE,W,J,1981-02-20,NaN,1945.0
622531,MICHAEL,SPIZZIRRI,2016-10-31,MALE,G,R,NaN,18265.0,1992.0


In [10]:
rd_df = remove_duplicates(dfu, id_cols).reset_index(drop=True)
print(rd_df.head())
rd_df.shape

  First.Name Last.Name Appointed.Date Gender Middle.Initial Middle.Initial2  \
0      AARON   ACEVEDO     2009-12-16   MALE              F             NaN   
1      AARON     ALLEY     2016-06-27   MALE              V             NaN   
2      AARON  ARMSTEAD     2005-12-05   MALE              P             NaN   
3      AARON     BURNS     2013-06-03   MALE              C             NaN   
4      AARON  CARRANZA     2004-11-29   MALE              R             NaN   

  Resignation.Date  Current.Star  Birth.Year  
0              NaN        7792.0      1982.0  
1              NaN        9516.0      1985.0  
2              NaN       13768.0      1975.0  
3              NaN        4534.0      1983.0  
4              NaN        1140.0      1980.0  


(32925, 9)

In [8]:
rd_df.insert(0, 'NID', rd_df.index + 1)

In [9]:
#out_df = df.merge(rd_df[[uid] + id_cols], on = id_cols, how='inner')
#out_df.head()

In [11]:
kd_df = keep_duplicates(dfu, id_cols)
print(kd_df.shape)
kd_df

(54, 9)


,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Middle.Initial2,Resignation.Date,Current.Star,Birth.Year
52252,ANN,HERR,2002-06-11,FEMALE,M,NaN,2010-06-16,NaN,1966.0
52263,ANN,HERR,2002-06-11,FEMALE,M,NaN,NaN,NaN,1966.0
61130,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,A,NaN,NaN,8280.0,1970.0
394495,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,NaN,NaN,NaN,NaN,1970.0
523,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,NaN,6673.0,1971.0
541,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,NaN,NaN,1971.0
101509,DAVID,RAMOS,1994-07-05,MALE,NaN,NaN,2009-10-21,NaN,1959.0
101515,DAVID,RAMOS,1994-07-05,MALE,R,NaN,NaN,8560.0,1969.0
136829,EDDIE,YOSHIMURA,1986-11-17,MALE,M,NaN,NaN,2334.0,1957.0
136837,EDDIE,YOSHIMURA,1986-11-17,MALE,M,NaN,NaN,NaN,NaN


In [405]:
def resolve_conflicts(df, id_cols, resolve_cols, uid, starting_uid, , temp_fillna = -9999):
    out_df = pd.DataFrame()
    df = df.reset_index(drop=True)
    group_df = df.fillna(temp_fillna).groupby(id_cols, as_index=False)
    for key, group in group_df:
        group = group.reset_index(drop=True)
        conflicts = 0
        for col in compare_cols:
            non_nan = group[col].dropna()
            non_nan = non_nan[(non_nan != -999) & (non_nan != temp_fillna)]
            if len(non_nan.unique()) > 1:
                conflicts +=1
        if conflicts == 0:
            group.insert(0, temp_id, max_temp_id + 1)
            max_temp_id += 1
        else:
            group.insert(0, temp_i, max_temp_id + group.index + 1)
            max_temp_id += max(group.index) + 1
        out_df = out_df.append(group)
    out_df[out_df == temp_fillna] = np.nan
    return out_df

In [406]:
rc = resolve_conflicts(kd_df, id_cols,compare_cols, max(rd_df['NID']))

In [407]:
rc

,NID,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Resignation.Date,Current.Star
0,26095,ANN,HERR,2002-06-11,FEMALE,M,2010-06-16,-999
1,26095,ANN,HERR,2002-06-11,FEMALE,M,NaN,-999
0,26096,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,NaN,NaN,-999
1,26096,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,A,NaN,8280
0,26097,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,6673
1,26097,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,-999
0,26098,DAVID,RAMOS,1994-07-05,MALE,NaN,2009-10-21,-999
1,26098,DAVID,RAMOS,1994-07-05,MALE,R,NaN,8560
0,26099,EDDIE,YOSHIMURA,1986-11-17,MALE,M,NaN,2334
1,26099,EDDIE,YOSHIMURA,1986-11-17,MALE,M,NaN,-999


In [408]:
df2 = df.merge(rd_df.append(rc), on=id_cols + compare_cols, how='left')

In [409]:
assert (df2[df2['NID'].isnull()].shape[0] == 0), "Uh-oh, some rows do not have IDs"

In [410]:
rc[(rc['First.Name'] == 'DAVID') & (rc['Last.Name'] == 'ADCOCK') & (rc['Appointed.Date'] == '2002-09-30')]

,NID,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Resignation.Date,Current.Star
0,26097,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,6673
1,26097,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,-999


In [411]:
print(tdf.shape)
tdf[id_cols + compare_cols].drop_duplicates()

(0, 22)


,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Resignation.Date,Current.Star


In [395]:
rc

,NID,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Resignation.Date,Current.Star
0,26095,ANN,HERR,2002-06-11,FEMALE,M,2010-06-16,-999
1,26095,ANN,HERR,2002-06-11,FEMALE,M,NaN,-999
2,26097,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,NaN,NaN,-999
3,26097,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,A,NaN,8280
4,26099,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,6673
5,26099,DAVID,ADCOCK,2002-09-30,MALE,G,NaN,-999
6,26101,DAVID,RAMOS,1994-07-05,MALE,NaN,2009-10-21,-999
7,26101,DAVID,RAMOS,1994-07-05,MALE,R,NaN,8560
8,26103,EDDIE,YOSHIMURA,1986-11-17,MALE,M,NaN,2334
9,26103,EDDIE,YOSHIMURA,1986-11-17,MALE,M,NaN,-999


In [412]:
df2[df2['NID'].isnull()]

,Gender,Race,Appointed.Date,Award.Type,Incident.Start.Date,Current.Award.Status,Award.Request.Date,Incident.End.Date,Resignation.Date,Award.ID,...,Birth.Year,Last.Promotion.Date,Requester.Full.Name,Ceremony.Date,Tracking.NO,Last.Name,Suffix.Name,First.Name,Middle.Initial,NID


In [413]:
nids = sorted(df2['NID'].unique())

In [415]:
assert nids[-1] == len(nids), print ('hey! got the wrong nids')

In [416]:
len(nids)

26124

In [417]:
max(nids)

26124

In [418]:
nids[-1]

26124